<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/2_02_ParametrosRespuesta_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temperatura, Máx. Tokens y Streaming (Gemini)

> Versión adaptada a la API de **Google Gemini** y traducida al español.


¡Bienvenido de nuevo!

En secciones anteriores hablamos de los **roles** que normalmente se usan al conversar con un modelo (por ejemplo, instrucciones tipo *system* y mensajes del *usuario*), y vimos cómo esas instrucciones pueden cambiar drásticamente el estilo del asistente (por ejemplo, un bot sarcástico).

En esta lección trabajaremos tres parámetros que afectan la respuesta del modelo:

<ul>
  <li><b>Máximo de tokens de salida</b> (longitud de la respuesta),</li>
  <li><b>Temperatura</b> (aleatoriedad / creatividad), y</li>
  <li><b>Streaming</b> (ver la respuesta “en vivo”, por partes).</li>
</ul>

Empecemos por la longitud (tokens de salida).  
En Gemini, el parámetro equivalente se configura como <code>max_output_tokens</code> dentro de <code>GenerationConfig</code>.


In [ ]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")


## 0) Instalación (solo la primera vez en Colab)
Ejecuta esta celda si estás en Google Colab o si no tienes instalada la librería.


In [ ]:
# =============================================================
# 🟦 1. Instalación y Configuración Inicial
# =============================================================
!pip install "langchain>=1,<2" langchain-core langchain-google-genai

## 3) Crear el modelo Gemini

> Nota: el nombre exacto del modelo puede variar según disponibilidad en tu cuenta. Si uno falla, prueba con `gemini-2.5-flash`.


In [ ]:
# Ya esta deprecated. Se usa por mostrar un ejemplo. Se actualizará en un futuro
!pip -q install -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.generativeai.types import GenerationConfig

genai.configure(api_key=api_key)

MODEL_NAME = "gemini-2.5-flash"  # alternativo: "gemini-2.5-flash-lite"


## 4) Ejemplo 1: Máximo de tokens (salida)

Vamos a recrear la idea del bot “Marv”, con una personalidad sarcástica.

Pregunta del usuario:
*¿Podrías explicar brevemente qué es un agujero negro?*

Primero, permitamos una respuesta de hasta **250 tokens** de salida.


In [ ]:
# Configuración del modelo
system_instruction = (
    "Eres Marv, un chatbot que responde preguntas con un tono sarcástico (sin insultar). "
    "Aun así, procura ser correcto y claro."
)

marv = genai.GenerativeModel(
    model_name=MODEL_NAME,   # por ejemplo: "gemini-2.5-flash"
    system_instruction=system_instruction
)

prompt_usuario = "¿Podrías explicar brevemente qué es un agujero negro?"

config_250 = GenerationConfig(max_output_tokens=250

)

# Generación
respuesta = marv.generate_content(
    prompt_usuario,
    generation_config=config_250
)

# 👉 SOLO EL TEXTO
print(respuesta.text)

Como era de esperarse, nuestro bot sarcástico responde… a su manera 🙂

Ahora bajemos el límite a **50 tokens** y observemos el cambio.


In [ ]:
config_50 = GenerationConfig(max_output_tokens=50)

respuesta = marv.generate_content(prompt_usuario, generation_config=config_50)


In [ ]:
# 👉 SOLO EL TEXTO
print(respuesta.text)

Esta vez la respuesta es mucho más corta (y puede quedar incompleta). Así que hay que tener cuidado de no limitar demasiado el modelo.


Volvamos a **250 tokens** y pasemos a **temperatura**.

La temperatura suele tomar valores entre 0 y 2 (dependiendo del proveedor/SDK). Valores mayores aumentan la aleatoriedad. Aunque muchas veces el valor por defecto funciona bien, conviene probar los extremos.

Ahora quitaremos el “sarcamo” y haremos un bot **educativo**. Ajusta la temperatura a **0** para una salida más determinista.


In [ ]:
profe = genai.GenerativeModel(
    model_name=MODEL_NAME,
    system_instruction="Eres un profesor universitario. Explicas con claridad y con ejemplos simples."
)

config_temp0 = GenerationConfig(
    temperature=0.0,
    max_output_tokens=250
)

respuesta = profe.generate_content(prompt_usuario, generation_config=config_temp0)


In [ ]:
print(respuesta.text)

Obtenemos una respuesta informativa.

Ahora vuelve a ejecutar la celda anterior (la que define `respuesta`) y observa el resultado.  
Con **temperatura baja**, las respuestas tienden a ser muy parecidas (aunque no idénticas), lo cual es útil en contextos educativos o cuando buscas consistencia.


Ahora subamos la temperatura al máximo (por ejemplo, **2.0**) y comparemos.


In [ ]:
config_temp2 = GenerationConfig(
    temperature=2.0,
    max_output_tokens=250
)

respuesta = profe.generate_content(prompt_usuario, generation_config=config_temp2)


In [ ]:
print(respuesta.text)

Probablemente notarás que la respuesta se vuelve más creativa… y también más propensa a desviarse o a “rellenar” con cosas menos útiles. En general, temperaturas muy altas rara vez ayudan en tareas de explicación técnica.

Volvamos a temperatura 0.


In [ ]:
respuesta = profe.generate_content(prompt_usuario, generation_config=config_temp0)
print(respuesta.text)

## 5) Semilla (seed) para mayor reproducibilidad

Otro parámetro que busca aumentar la consistencia es **seed** (semilla), similar a lo que hacemos en ML para reproducibilidad.

En LLMs la determinación **no está garantizada** al 100%, pero un `seed` fijo puede ayudar a que las salidas sean lo más parecidas posible.

> Ojo: el soporte de `seed` puede variar por versión del SDK / backend. Por eso lo aplicaremos con un enfoque “seguro”.


In [ ]:
# Intentamos fijar seed (si el backend/SDK lo soporta)
def make_config_temp0_seed(seed: int = 365):
    try:
        return GenerationConfig(
            temperature=0.0,
            max_output_tokens=250,
            seed=seed
        )
    except TypeError:
        # Si tu versión no soporta 'seed', caemos a configuración sin seed
        return GenerationConfig(
            temperature=0.0,
            max_output_tokens=250
        )

config_temp0_seed = make_config_temp0_seed(365)

respuesta = profe.generate_content(prompt_usuario, generation_config=config_temp0_seed)
print(respuesta.text)

## 6) Streaming (respuesta en vivo)

Una manera de hacer un chatbot más “reactivo” es imprimir la salida **mientras se genera**, en lugar de esperar a que termine.

En Gemini, puedes usar `stream=True` para obtener un iterador con “trozos” (chunks) de la respuesta.


In [ ]:
stream = profe.generate_content(
    prompt_usuario,
    generation_config=config_temp0_seed,
    stream=True
)

stream

Lo importante es que este objeto se puede iterar con un `for`.
Primero, veamos los objetos (chunks) que llegan:


In [ ]:
for chunk in stream:
    print(chunk)


Ahora hagamos *streaming* del texto: (vuelve a ejecutar la celda que define `stream` para regenerarlo)


In [ ]:
stream = profe.generate_content(
    prompt_usuario,
    generation_config=config_temp0_seed,
    stream=True
)

for chunk in stream:
    # Cada chunk suele traer una parte del texto
    if getattr(chunk, "text", None):
        print(chunk.text, end="")


¡Listo! Has hecho streaming de la respuesta en pantalla 🙂  

## Cierre

Con esto termina esta introducción corta a la **API de Gemini** (en el estilo del notebook original).

La gracia de trabajar con una API es que te da control fino sobre:
- longitud de salida,
- creatividad,
- consistencia,
- y experiencia de usuario (streaming).

En las siguientes secciones (por ejemplo, con **LangChain** / **LangGraph**) podrás construir flujos más complejos: herramientas, memoria,agentes, etc.


In [ ]:
# Fin del notebook ✅